<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 3 Central Limit Theorem ###

In [ ]:
### standard imports
import numpy as np
import pandas as pd

%matplotlib inline  
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True

## Let's compare Binomial distribution with CLT approx Normal distribution

### scipy stats distributions

[scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html) has all the common distributions for you to generate samples from or calculate properties of a given distribution. Using normal distribution as an example, the following functions will be useful. Each function takes arguments which specifies the parameters for the distribution (for normal loc=mu, scale=sigma, and the default is standard normal), and some additional arguments. For other distributions, replace the name `norm` with the proper distribution name in the documentation. 

- `scipy.stats.norm.pdf(x, ...)`: additional argument x, the point(s) where you want to evaluate the pdf, has a log version. For discrete distributions, it is pmf instead of pdf.
- `scipy.stats.norm.cdf(x, ...)`: additional argument x, the point(s) where you want to evaluate the cdf, has a log version, has a log version.
- `scipy.stats.norm.ppf(q, ...)`: additional argument q, the probability where you want to evaluate the inverse cdf.
- `scipy.stats.norm.mean(...)`: returns mean of the distribution.
- `scipy.stats.norm.median(...)`: returns median of the distribution.
- `scipy.stats.norm.var(...)`: returns variance of the distribution.
- `scipy.stats.norm.std(...)`: returns standard deviation of the distribution.
- `scipy.stats.norm.expect(...)`: returns expected value of a function of the random variable with respect to the distribution.


In [ ]:
# example of binomial and normal
from scipy.stats import binom, norm

In [ ]:
# number of trials (flip a coin)
n = 10
# probability of success (result a head)
p = 0.5
# probability of getting 3 heads
binom.pmf(3, n, p)

In [ ]:
# value of cdf
# probability of getting # heads or fewer
binom.cdf(4, n, p)

In [ ]:
# value of inverse cdf
# getting how many heads or fewer corresponds to (closest to) probablity 0.3?
binom.ppf(0.3, n, p)

In [ ]:
# mean of binomial = n*p
binom.mean(n, p)

In [ ]:
# var of binomial = n*p*(1-p)
binom.var(n, p)

In [ ]:
# normal pdf with mean=0 and sd=1
norm.pdf(3, loc=0, scale=1)
# normal cdf with mean=0 and sd=1
norm.cdf(3, 0, 1)
# normal inverse cdf of 0.975
norm.ppf(0.975, 0, 1)

In [ ]:
# define an array from 0 to n, all the possible value for a binomial(n, p)
x = np.arange(0, n+1)
# binomial pmf evaluated for each x
bin_pmf = binom.pmf(x, n, p)
# binomial cdf evaluated for each x
bin_cdf = binom.cdf(x, n, p)

In [ ]:
# mu for Bernoulli
mu = p
# sigma square for Bernoulli
sigma_sq = p*(1-p)
sigma = np.sqrt(p*(1-p))

# sample mean is Binomal divided by n
x_hat = x/n

# by CLT x_hat is approx N(mu, sigma/sqrt(n))
z = (x_hat - mu)/(sigma /np.sqrt(n))
norm_app_pdf = norm.pdf(z, 0, 1)
norm_app_cdf = norm.cdf(z, 0, 1)

# same results using the normal before standardizing
#norm_app_pdf = norm.pdf(x_hat, mu, sigma/np.sqrt(n))
#norm_app_cdf = norm.cdf(x_hat, mu, sigma/np.sqrt(n))

In [ ]:
df = pd.DataFrame({'number of heads':np.arange(0, n+1), 'binom pmf':bin_pmf, 'norm CLT pdf':norm_app_pdf,
                      'binom cdf':bin_cdf, 'norm CLT cdf':norm_app_cdf})
df.head(50)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
fig.suptitle('Comparing Exact Binomial to Approx Normal by CLT')

ax1.scatter(x=df['number of heads'], y=df['binom pmf'], alpha=0.5, s=50, label='Binomial exact')
ax1.scatter(x=df['number of heads'], y=df['norm CLT pdf'], alpha=0.5, s=50, label='Normal by CLT')
ax1.set(xlabel='Number of Heads', ylabel='PMF / PDF ', ylim=[0,1])
ax1.legend()

ax2.scatter(x=df['number of heads'], y=df['binom cdf'], alpha=0.5, s=50, label='Binomial exact')
ax2.scatter(x=df['number of heads'], y=df['norm CLT cdf'], alpha=0.5, s=50, label='Normal by CLT')
ax2.set(xlabel='Number of Heads', ylabel='CDF ')
ax2.legend();

## Let's use some flight delay data to verify CLT
The data is a subset of [SFO Gate and Stand Assignment Information Data](https://data.sfgov.org/Transportation/SFO-Gate-and-Stand-Assignment-Information/chfu-j7tc)

In [ ]:
# data only contains 4 columns, delay in min
united = pd.read_csv('united_summer2015.csv')
united

In [ ]:
# use histogram to visualize distribution of delay
# sns.distplot(united['Delay'], kde=False);
plt.hist(united['Delay'], bins=40)
plt.title('Histogram of flights delay summer 2015')
plt.xlabel('Flights delay in min');

In [ ]:
delays = united['Delay']
# calculate mean and sd of delay
mean_delay = np.mean(delays)
sd_delay = np.std(delays)

print("flights delay mean is", mean_delay)
print("flights delay standard deviation is", sd_delay)

In [ ]:
# A function to generate many sample means given a fixed sample size
def united_sample_mean(sample_size, rep=10000):
    """Simulate sample mean:
    data: united flight delay
    sample_size: sample size
    rep: number samples to generate from the data
    returns list of sample means
    """
    # an empty list as place holder to save all simulated sample means
    sample_means = []
    for i in np.arange(rep):
        # each time take out a sample with fixed sample_size
        sampled_flights = united.sample(sample_size)
        # calculate one sample mean
        one_sample_mean = np.mean(sampled_flights['Delay'])
        # save the new sample mean by appending to the list
        sample_means = np.append(sample_means, one_sample_mean)
    # return the entire list
    return sample_means

In [ ]:
sample_size = 400
means_400 = united_sample_mean(sample_size)
# this should return 10000 sample means!
means_400

In [ ]:
# plot the 10000 sample means in a histogram
# that is approx the sampling distribution of sample means with sample size 400
plt.hist(means_400, bins=20)
plt.title('Sample Means: Sample Size '+ str(sample_size))
plt.xlabel('Sample Mean')
print('Population Average: ', mean_delay);

In [ ]:
# average of the 10000 sample means (with sample size 400)
np.average(means_400)

In [ ]:
# standard deviation of the 10000 sample means (with sample size 400)
np.std(means_400)

In [ ]:
# standard deviation derived by CLT: 
# population standard deviation divided by square root of sample size
sd_delay / np.sqrt(400)

## Variability of the Sample Mean ##

In [ ]:
# repeat the same simulation, this time with sample size 900
sample_size = 900
means_900 = united_sample_mean(sample_size)

In [ ]:
# put the 2 simulation in a dataframe 
# 10000 rows each row is a simulated sample mean
means_df = pd.DataFrame({'sample mean of 400':means_400, 'sample mean of 900':means_900})
means_df

In [ ]:
# plot two histograms on the same plot
# what did you observe?
means_df.plot(kind='hist',bins = np.arange(10, 25, 0.5), alpha=0.4)
plt.title('Sampling distribution of sample means with different sample size')
plt.xlabel('Sample means');

In [ ]:
def plot_sample_means(sample_size, rep=10000):
    """
    simulate sample mean and make plot 
    call the function plot_sample_means
    no return value but plot histogram
    """
    means = united_sample_mean(sample_size, rep)

    # Display histogram for empirical distribution and print all relevant quantities
    plt.hist(means, bins=20)
    plt.xlabel('Sample Means')
    plt.title('Sample Size ' + str(sample_size))
    print("Sample size: ", sample_size)
    print("Population mean:", np.mean(united['Delay']))
    print("Average of sample means:", np.mean(means))
    print("Population SD:", np.std(united['Delay']))
    print("SD of sample means:", np.std(means))
    print ("SD of sample means by CLT:", (np.std(united['Delay'])/np.sqrt(sample_size)))

In [ ]:
# try with sample size 100, 400 and 625
plot_sample_means(100)

In [ ]:
# 8 different sample size
sample_sizes = np.arange(50, 401, 50)

sd_of_sample_means = []

# loop over sample size
for n in sample_sizes:
    means = []
    # for each sample size n, generate 10000 sample means
    for i in np.arange(10000):
        # generate one sample mean then append
        one_mean = np.mean(united.sample(n)['Delay'])
        means = np.append(means, one_mean)
    # calculate sd of the 10000 sample mean
    one_sd = np.std(means)
    sd_of_sample_means = np.append(sd_of_sample_means, one_sd)
# sd_of_sample_means contains 8 sd for the 8 sample sizes
# from total of 8x10000 sample means

In [ ]:
# put everything in a dataframe for easy visualization
sd_comparison = pd.DataFrame(
    {'Sample Size n': sample_sizes,
    'SD of 10,000 Sample Means': sd_of_sample_means,
    'Population_SD/sqrt(n)': sd_delay/np.sqrt(sample_sizes) }
)

In [ ]:
# 3 columns: 
# sample size
# SD of sample mean calculatd by simulation
# SD of sample mean calculated by CLT
sd_comparison

In [ ]:
# make a plot of the above dataframe
plt.scatter(x=sd_comparison['Sample Size n'], y=sd_comparison['SD of 10,000 Sample Means'], alpha=0.5, s=50, label='SD of sample mean')
plt.scatter(x=sd_comparison['Sample Size n'], y=sd_comparison['Population_SD/sqrt(n)'], alpha=0.5, s=50, label='SD of sample mean by CLT')
plt.title('Comparing SD of sample mean to CLT')
plt.xlabel('Sample size')
plt.ylabel('SD of sample mean')
plt.legend();

## In your homework, try the following simulations yourself:
-  Can you design a simulation to show CLT is also true for a population distribution that is discrete?
- Can you design a simulation to show CLT is NOT true for other statistics that is not sample mean?